In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import os
os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2018/bin/x86_64-darwin'
plt.rc('text', usetex=True)
plt.rc('font', weight='bold')
import re

## Import Data

In [ ]:
fnames = ['/Users/hkromer/02_PhD/02_Data/08.Microwave.PermanentMagnets/Messung01_magneticField.txt', '/Users/hkromer/02_PhD/02_Data/08.Microwave.PermanentMagnets/Messung02_magneticField.txt']

In [ ]:
dfs = []
for fname in fnames:
    df = pd.read_csv(fname, skiprows=0, delimiter='\t')
    df.iloc[:,1] = df.iloc[:,1] * 1000 # convert to Gauss
    dfs.append(df)


## Plot

In [ ]:
plt.plot(dfs[0].iloc[:,0], dfs[0].iloc[:,1], marker='o',label='Messung 01', alpha=0.4)

plt.plot(dfs[1].iloc[:,0], dfs[1].iloc[:,1], marker='s', label='Messung 02', alpha=0.4)
xlim = plt.gca().get_xlim()
plt.plot([-10,200], [875, 875], color='red', linestyle='dashed', label='ECR, 875 Gauss')
plt.legend()
plt.grid()
plt.xlabel('Distance from iron frontplate [mm]')
plt.ylabel('Magnetic field (Bz) [Gauss]')
ylim = plt.gca().get_ylim()
plt.ylim(0, ylim[1])
plt.xlim(xlim)
plt.show()

In [ ]:
dfs[1]

# COMSOL Model
- Iron frontplate is at position 0 mm
- Plasma cup extends into the minus direction
- Data is in /Users/hkromer/02_PhD/02_Data/01_COMSOL/04.Microwave/BfieldData/02.damaged.Br1.10.relPermIron1000.1mmHole.csv and /Users/hkromer/02_PhD/02_Data/01_COMSOL/04.Microwave/BfieldData/02.damaged.Br1.03.relPermIron1000.1mmHole.csv
    - the data is for X {-100, 10, 1}, Y {-5, 5, 0.2}, Z {-5, 5, 0.2}
    - header: X,Y,Z, mfnc.normB (mT)
- From the manufacturer the magnetic flux density in the magnets is typically 1.10T, but certified to be higher than 1.03T
- Original simulation are in /Users/hkromer/02_PhD/02_Data/01_COMSOL/04.Microwave/03.undamaged.Br1.10.relPermIron1000.1mmHole_out.mph and /Users/hkromer/02_PhD/02_Data/01_COMSOL/04.Microwave/03.undamaged.Br1.03.relPermIron1000.1mmHole_out.mph

## Treat each y and z value separately
- for each x value find the minimum and maximum B field (depending on the y, z values)
- store in a new column
- plot this around the centerline (y=z=0)

## Transform

In the measurement the x=0mm is the frontplate, same is for the COMSOL. But in the COMSOL simulation the x goes in - direction, so everything has to be multiplied with -1.

The values at x =0 is not physical

## Take experimental error into account
- assumption is that ±1.5mm for the position for x<40 mm, and ±3mm for further away

In [ ]:
fnames = ['/Users/hkromer/02_PhD/02_Data/01_COMSOL/04.Microwave/BfieldData/02.damaged.Br1.03.relPermIron1000.1mmHole.csv', '/Users/hkromer/02_PhD/02_Data/01_COMSOL/04.Microwave/BfieldData/02.damaged.Br1.10.relPermIron1000.1mmHole.newIron.csv']
output_folder ='/Users/hkromer/polybox/Thesis/Chapters/FurtherNeutronGeneratorUpgrades/Figures/Magnetic_fiel_measurement_simulation/'
dfs_comsol = []
dfs_comsol_maxmin = []
dfs_comsol_centerline = []
dfs_comsol_2mmline = []


def find_max_Bfield(df):
    mymax = df['normB_Gauss'].max()
    mymin = df['normB_Gauss'].min()
    return pd.Series({'normB_Gauss_max': mymax, 'normB_Gauss_min': mymin})


# load the files and append into a list of dataframes
for fname in fnames:
    df_comsol = pd.read_csv(fname, skiprows=8)
    df_comsol.columns = ['x', 'y', 'z', 'normB_mT']
    df_comsol['normB_Gauss'] = df_comsol['normB_mT'] * 10
    print(df_comsol.info())
    dfs_comsol.append(df_comsol)
    
    

    df_comsol = df_comsol.sort_values(by=['x'])


    df_comsol_maxmin = df_comsol.groupby('x').apply(lambda x: find_max_Bfield(x))
    # transform
    df_comsol_maxmin['x'] = df_comsol_maxmin.index * -1.0
    df_comsol_maxmin.set_index('x', inplace=True) 
    df_comsol_maxmin.drop([0.0, -1.0], inplace=True)
    dfs_comsol_maxmin.append(df_comsol_maxmin)
    print(df_comsol_maxmin.describe())
    
    df_comsol_centerline = df_comsol[ (df_comsol['y']==0) & (df_comsol['z']==0) ]
    df_comsol_centerline['x'] = df_comsol_centerline['x'] * -1.0
    df_comsol_centerline.set_index('x', inplace=True) 
    dfs_comsol_centerline.append(df_comsol_centerline)
    
    df_comsol_2mmline = df_comsol[ (df_comsol['y']==-5) & (df_comsol['z']==5) ]
    df_comsol_2mmline['x'] = df_comsol_2mmline['x'] * -1.0
    df_comsol_2mmline.set_index('x', inplace=True) 
    dfs_comsol_2mmline.append(df_comsol_2mmline)

## Experiment and Simulation in comparison

In [ ]:
smallerror = 1.5
bigerror = 3
def add_error(row, smallerror, bigerror):
    if row < 40:
        return pd.Series({'x_plus': row+smallerror, 'x_minus': row-smallerror})
    else:
        return pd.Series({'x_plus': row+bigerror, 'x_minus': row-bigerror})

    
a = dfs[1].iloc[:,0].apply(lambda x: add_error(x, smallerror, bigerror))
df_error = pd.concat([a, dfs[1]], axis=1)

In [ ]:
# take error into account
from scipy.interpolate import interp1d
f1 = interp1d(df_error['x_plus'], df_error.iloc[:,3], kind='cubic', fill_value="extrapolate")
f2 = interp1d(df_error['x_minus'], df_error.iloc[:,3], kind='cubic', fill_value="extrapolate")

In [ ]:
import os, matplotlib
os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2018/bin/x86_64-darwin'
plt.rc('text', usetex=True)
plt.rc('font', weight='bold')
matplotlib.rcParams['mathtext.fontset'] = 'custom'
matplotlib.rcParams['mathtext.rm'] = 'Arial'
matplotlib.rcParams['mathtext.it'] = 'Arial:italic'
matplotlib.rcParams['mathtext.bf'] = 'Arial:bold'
matplotlib.rcParams['mathtext.tt'] = 'Arial'
matplotlib.rcParams['mathtext.cal'] = 'Arial'
matplotlib.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']

fig = plt.figure(figsize=(8,6))
fs = 20
# plt.plot(dfs[0].iloc[:,0], dfs[0].iloc[:,1], marker='o',label='Messung 01', alpha=0.75)



plt.plot(dfs_comsol_centerline[1].index, dfs_comsol_centerline[1].loc[:,'normB_Gauss'], color='darkorange', label='Simulation at y=z=0 mm',  linewidth=2)
plt.plot([38.5, 38.5], [200, 1200], '--', color='black')
plt.plot([0, 0], [200, 1200], '--', color='black')
# plt.plot(dfs_comsol_2mmline[1].index, dfs_comsol_2mmline[1].loc[:,'normB_Gauss'], color='darkred', label='Simulation, 1.10 T 2mm line',  linewidth=1)
# plt.plot(dfs_comsol_centerline[0].index, dfs_comsol_centerline[0].loc[:,'normB_Gauss'], color='darkgreen', label='Simulation 1.03 T centerline',  linewidth=1)
# plt.plot(dfs_comsol_maxmin[1].index, dfs_comsol_maxmin[1].loc[:,'normB_Gauss_min'], color='darkred', label='Simulation 1.10 T max',  linewidth=1)
# plt.plot(dfs_comsol_maxmin[0].index, dfs_comsol_maxmin[0].loc[:,'normB_Gauss_min'], color='darkorange', label='Simulation 1.03 T min',  linewidth=1)
# plt.fill_between(dfs_comsol_maxmin[0].index, dfs_comsol_maxmin[1].loc[:,'normB_Gauss_min'], dfs_comsol_maxmin[0].loc[:,'normB_Gauss_min'], alpha=0.3,color='darkred')

plt.plot(dfs[1].iloc[:,0], dfs[1].iloc[:,1], marker='o', color='darkblue', markeredgecolor='darkblue', markersize=8, label='Experiment')
plt.fill_between(dfs[1].iloc[:,0], f1(dfs[1].iloc[:,0]), f2(dfs[1].iloc[:,0]), alpha=0.25, color='darkblue', linewidth=0)


xlim = plt.gca().get_xlim()
plt.plot([-10,200], [875, 875], color='red', linestyle='dashed', label='ECR (875 G)')
plt.legend(fontsize=fs-6)
plt.grid()
plt.xlabel(r'\textbf{Distance from ferritic frontplate [mm]}', fontsize=fs, labelpad=2)
plt.ylabel(r'\textbf{Magnetic field density [G]}', fontsize=fs, labelpad=2)
ylim = plt.gca().get_ylim()
plt.gca().tick_params('x', colors='black', labelsize=fs-2)	
plt.gca().tick_params('y', colors='black', labelsize=fs-2)
plt.text(36, 400, 'Bottom of plasma cup', fontsize=14, rotation=90)
plt.text(1, 400, 'Plasma aperture', fontsize=14, rotation=90)

plt.ylim(200, 1200)
plt.xlim(-2,85)
plt.tight_layout()
plt.savefig(f'{output_folder}Magnetic_fiel_measurement_simulation.pdf', dpi=900)
plt.show()

## Plot

In [ ]:
plt.plot(df_comsol_maxmin.index, df_comsol_maxmin['normB_Gauss_max'], color='red', label='max', alpha=0.4)
plt.plot(df_comsol_maxmin.index, df_comsol_maxmin['normB_Gauss_min'], color='red', label='min', alpha=0.4)

xlim = plt.gca().get_xlim()
plt.plot([-10,200], [875, 875], color='red', linestyle='dashed', label='ECR, 875 Gauss')
plt.legend()
plt.grid()
plt.xlabel('Distance from iron frontplate [mm]')
plt.ylabel('Magnetic field (Bz) [Gauss]')

# plt.ylim(0, ylim[1])
plt.xlim(xlim)
plt.show()

## Experiment and Simulation in comparison

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(dfs[0].iloc[:,0], dfs[0].iloc[:,1], marker='o',label='Messung 01', alpha=0.75)

plt.plot(dfs[1].iloc[:,0], dfs[1].iloc[:,1], marker='s', label='Messung 02', alpha=0.75)

xlim = plt.gca().get_xlim()
plt.plot(df_comsol_maxmin.index, df_comsol_maxmin['normB_Gauss_max'], color='red', label='max', alpha=0.4)
plt.plot(df_comsol_maxmin.index, df_comsol_maxmin['normB_Gauss_min'], color='green', label='min', alpha=0.4)
# plt.plot([-10,200], [875, 875], color='red', linestyle='dashed', label='ECR, 875 Gauss')
plt.legend()
plt.grid()
plt.xlabel('Distance from iron frontplate [mm]')
plt.ylabel('Magnetic field (Bz) [Gauss]')
ylim = plt.gca().get_ylim()
plt.ylim(0, 1200)
plt.xlim(xlim)
plt.show()

## compare different relative iron plate values

In [ ]:
path = '/Users/hkromer/02_PhD/02_Data/01_COMSOL/04.Microwave/BfieldData/'
fnames = os.listdir(path)
fnames = [f'{path}{f}' for f in fnames if f.endswith('csv')]
fnames

In [ ]:
dfs = []
def find_max_Bfield(df):
    mymax = df['normB_Gauss'].max()
    mymin = df['normB_Gauss'].min()
    return pd.Series({'normB_Gauss_max': mymax, 'normB_Gauss_min': mymin})

for fname in fnames:
    ID = re.findall(r'Iron(\d+)\.', fname)[0]
    df_comsol = pd.read_csv(fname, skiprows=8)
    df_comsol.columns = ['x', 'y', 'z', 'normB_mT']
    df_comsol['normB_Gauss'] = df_comsol['normB_mT'] * 10
    df_comsol = df_comsol.sort_values(by=['x'])

    df_comsol_maxmin = df_comsol.groupby('x').apply(lambda x: find_max_Bfield(x))
    df_comsol_maxmin['ID'] = ID
    dfs.append(df_comsol_maxmin)

In [ ]:
for df_comsol_maxmin in dfs:
    ID = df_comsol_maxmin.ID.unique()[0]
    plt.plot(df_comsol_maxmin.index, df_comsol_maxmin['normB_Gauss_max'], label=f'{ID}_max', alpha=0.4)

xlim = plt.gca().get_xlim()
# plt.plot([-10,200], [875, 875], color='red', linestyle='dashed', label='ECR, 875 Gauss')
plt.legend()
plt.grid()
plt.xlabel('Distance from iron frontplate [mm]')
plt.ylabel('Magnetic field (Bz) [Gauss]')

# plt.ylim(0, ylim[1])
plt.xlim(xlim)
plt.show()

In [ ]:
for df_comsol_maxmin in dfs:
    ID = df_comsol_maxmin.ID.unique()[0]
    plt.plot(df_comsol_maxmin.index, df_comsol_maxmin['normB_Gauss_min'], label=f'{ID}_min', alpha=0.4)

xlim = plt.gca().get_xlim()
# plt.plot([-10,200], [875, 875], color='red', linestyle='dashed', label='ECR, 875 Gauss')
plt.legend()
plt.grid()
plt.xlabel('Distance from iron frontplate [mm]')
plt.ylabel('Magnetic field (Bz) [Gauss]')

plt.ylim(915,917)
plt.xlim(xlim)
plt.show()